## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Display the updated dataframe
application_df.head()

In [ ]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500  
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts[classification_counts > 1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 100 
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the function to create the model with hyperparameters
def create_model(optimizer='adam', learning_rate=0.001, dropout_rate=0.3, units_first_layer=256, units_second_layer=128, units_third_layer=64):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=units_first_layer, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_dim=X_train_scaled.shape[1]))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(units=units_second_layer, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(units=units_third_layer, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    optimizer_instance = tf.keras.optimizers.get(optimizer)
    optimizer_instance.learning_rate = learning_rate
    model.compile(optimizer=optimizer_instance, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Define the function to train and evaluate the model
def train_and_evaluate(optimizer, learning_rate, dropout_rate, units_first_layer, units_second_layer, units_third_layer, epochs=100):
    nn = create_model(optimizer, learning_rate, dropout_rate, units_first_layer, units_second_layer, units_third_layer)
    history = nn.fit(X_train_scaled, y_train, epochs=epochs, batch_size=32, validation_split=0.2, verbose=1)
    model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
    return history, model_accuracy, nn

In [ ]:
# Model with Adam optimizer and reduced complexity
history_adam_optimized, adam_optimized_accuracy, nn_adam_optimized = train_and_evaluate('adam', 0.0005, 0.3, 256, 128, 64, 100)


In [ ]:
# Model with RMSprop optimizer and reduced complexity
history_rmsprop_optimized, rmsprop_optimized_accuracy, nn_rmsprop_optimized = train_and_evaluate('rmsprop', 0.0005, 0.3, 256, 128, 64, 100)


In [ ]:
# Model with Adam optimizer, dropout, and reduced complexity
history_adam_dropout_optimized, adam_dropout_optimized_accuracy, nn_adam_dropout_optimized = train_and_evaluate('adam', 0.0005, 0.3, 256, 128, 64, 100)


In [ ]:
# Evaluate the models using the test data
print(f"Adam Optimized Model Accuracy: {adam_optimized_accuracy}")
print(f"RMSprop Optimized Model Accuracy: {rmsprop_optimized_accuracy}")
print(f"Adam with Dropout Optimized Model Accuracy: {adam_dropout_optimized_accuracy}")


In [ ]:

# Plotting accuracy for each model
import matplotlib.pyplot as plt

def plot_history(history, title):
    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.plot(history.history['val_accuracy'], label='val accuracy')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

plot_history(history_adam_optimized, 'Adam Optimized Model Accuracy')
plot_history(history_rmsprop_optimized, 'RMSprop Optimized Model Accuracy')
plot_history(history_adam_dropout_optimized, 'Adam with Dropout Optimized Model Accuracy')

In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')